In [ ]:
# 수도권 전철 노선도에 대한 그래프(GML)를 만듭니다.

In [1]:
# 지하철 노선도 페이지를 로드합니다.
# 서울교통공사 사이버스테이션 자료를 참조하였습니다.
# HTML DOM 로드를 위해 Selenium을 사용하였습니다.
# 지하철 그래프를 생성하는 라이브러리로 networkx를 사용하였습니다.

In [3]:
from selenium import webdriver
import lxml.html
import os

driver = webdriver.Edge(os.getenv('LOCALAPPDATA')+'/Programs/Python/edgedriver_win64/msedgedriver.exe')
driver.get('http://www.seoulmetro.co.kr/kr/cyberStation.do')
html = driver.page_source

tree = lxml.html.fromstring(html)

In [4]:
# 웹 페이지 중 지하철 노선도를 나타내는 요소는 class명이 subway-map인 div 요소입니다.
# 이후 요소 추출 시 중복 조건에 따른 오류를 방지하기 위해 이를 헤더 요소로 사용합니다.

In [5]:
subway_map_div_element = tree.xpath('//div[@class="subway-map"]')[0]

In [6]:
# 역 ID, 노선 번호, 역명을 추출합니다.
# 헤더 요소의 자식 중 class명이 label-group인 g 요소는 각 역에 표시되는 역명의 집합을 나타냅니다.
# g 요소의 자식인 각각의 text 요소의 class는 label S[역 ID 1] L[노선 번호 1] S[역 ID 2] L[노선 번호 2]...의 형식을 가집니다.
# 또한 text 요소의 자식인 tspan 요소의 값은 역명을 나타냅니다. tspan이 두 개 이상일 경우 역명은 각각의 concatenation입니다.
# 이를 각각 추출하여 딕셔너리 변수를 생성합니다.

In [14]:
import re
from itertools import repeat, chain
from collections import defaultdict

id_no_dict = {} # 역 ID와 노선 번호의 대응 관계
id_name_dict = {} # 역 ID와 역명의 대응 관계

interchange_list = [] # 환승역, 각 역 ID의 리스트를 자식 요소로 하는 리스트

label_g_element = subway_map_div_element.xpath('//g[@class="label-group"]')[0]
for text_element in label_g_element.cssselect('text'):
    class_tag = text_element.attrib['class']
    station_name = ''.join(tspan_element.text.strip() for tspan_element in text_element.cssselect('tspan'))

    # class_tag를 각각의 역 ID, 노선 번호로 추출합니다.
    # 역 ID는 숫자와 알파벳 대문자로 이루어진 네 자리 문자열, 노선 번호는 한 자리 또는 두 자리 문자열입니다.
    # label S[역 ID 1] L[노선 번호 1] S[역 ID 2] L[노선 번호 2]...의 형식을 가집니다.
    id_no_list = re.findall('S([0-9A-Z]{4}) L([0-9A-Z]+)', class_tag)
    id_no_dict.update(id_no_list)
    id_list, no_list = zip(*id_no_list)
    id_name_dict.update(zip(id_list, repeat(station_name)))

    # # 노선 번호가 두 개 이상인 것은 환승역입니다(역 ID가 두 개인 것은 지선일 수 있음).
    no_set = set(no_list)
    if len(no_set) >= 2:
        interchange_list.append(id_list)
        
id_interchange_dict = defaultdict(bool)
id_interchange_dict.update({station_id: True for station_id in chain.from_iterable(interchange_list)}) # 역 ID와 환승역 여부의 대응 관계

# 일반적인 사용 용례 상 예외를 설정합니다(20201221 기준).
id_name_dict['4926'] = '사우'

In [15]:
# 그래프를 생성할 정점과 간선을 추출합니다.
# 그래프의 정점은 역, 간선은 두 역간의 연결입니다.
# 헤더 요소의 자식 중 class명이 line인 g 요소는 각 노선별 역을 연결한 path들을 자식으로 갖는 요소입니다.
# 각 path 요소들의 class는 path L[노선 번호] P[역 ID 1][역 ID 2] P[역 ID 2][역 ID 1]의 형식을 가집니다.
# 만일 class 형식이 path L[노선 번호] 인 경우, 실제 연결이 아닙니다.
# 따라서 역 ID 1과 역 ID 2를 추출하면 됩니다.

In [17]:
from itertools import combinations

edge_dict = dict() # 두 정점을 잇는 간선과 간선이 대표하는 노선의 대응 관계, 값이 빈 문자열일 경우 환승역간의 관계입니다.

for graph_element in subway_map_div_element.xpath('//g[@class="line"]'):
    for path_element in graph_element.cssselect('path'):
        class_tag = path_element.attrib['class']
        try:
            id1, id2 = re.findall('path L[0-9A-Z]+ P([0-9A-Z]{4})([0-9A-Z]{4}) P\\2\\1', class_tag)[0]
            edge_dict[(id1, id2)] = id_no_dict[id1] # 간선을 추가합니다.
        except Exception: # 형식이 맞지 않는 경우 올바른 연결이 아닙니다.
            continue

# 환승역간의 연결을 처리합니다.
edge_dict.update({k: '' for k in chain.from_iterable(list(combinations(interchange, 2)) for interchange in interchange_list)})

In [18]:
# 노선 번호와 노선명을 대응하는 딕셔너리를 생성합니다.
# 웹 페이지 중 노선에 대한 설명을 나타내는 요소는 class명이 mapInfo인 div 요소입니다.
# 이것의 내부 요소 중 li 요소는 노선 번호(data-line-id)와 노선명(자식 span 요소의 text)에 대한 정보를 가집니다.

In [19]:
no_name_dict = dict() # 노선 번호와 노선명을 대응하는 딕셔너리입니다.

map_info_div_element = tree.xpath('//div[@class="mapInfo"]')[0]
for li_element in map_info_div_element.cssselect('li'):
    line_no = li_element.attrib['data-line-id']
    line_name = li_element.cssselect('span')[0].text

    no_name_dict[line_no] = line_name

# 일반적인 사용 용례 상 예외를 설정합니다(20201221 기준).
no_name_dict.update({
    'S': '신분당선',
    'K': '경의중앙선',
    'E': '에버라인',
    'W': '우이신설선',
    'KP': '김포골드라인',
    })

In [20]:
# 노선 번호에 따른 표기 색을 설정합니다.

In [21]:
no_color_dict = {
    '1': '#0052A4',
    '2': '#00A84D',
    '3': '#EF7C1C',
    '4': '#00A4E3',
    '5': '#996CAC',
    '6': '#CD7C2F',
    '7': '#747F00',
    '8': '#E6186C',
    '9': '#BDB092',
    'I': '#759CCE',
    'I2': '#F5A251',
    'B': '#FABE00',
    'S': '#D4003B',
    'K': '#77C4A3',
    'A': '#0090D2',
    'G': '#178C72',
    'U': '#FB8100',
    'E': '#56AB2D',
    'KK': '#005BAC',
    'W': '#DBD350',
    'SH': '#8FC31F',
    'KP': '#AD8605',
    }

In [23]:
# 각 역의 좌표를 얻습니다.
# 좌표는 카카오로컬 API를 사용하였으며, 검색량이 많으므로 병렬 처리를 이용합니다.
# 병렬 처리의 원활한 작동을 위해 Jupyter Notebook이 아닌 외부 .py 파일로 함수를 만들었습니다.
# multiprocessing_worker 모듈의 get_coord_by_keyword 함수를 사용합니다.

In [25]:
from multiprocessing import Pool, cpu_count
from multiprocessing_worker import get_coord_by_keyword

id_coord_dict = dict()

query_list = (f'{id_name_dict[station_id]}역 {no_name_dict[id_no_dict[station_id]]}' for station_id in id_no_dict.keys()) # ex. '시청역 2호선'

pool = Pool(processes=cpu_count())
coord_list = pool.map(get_coord_by_keyword, query_list)

pool.close()
pool.join()

id_coord_dict.update(zip(id_no_dict.keys(), coord_list))

In [26]:
# 딕셔너리를 기반으로 그래프를 생성하고 저장합니다.

In [36]:
import networkx as nx

G = nx.Graph()

G.add_nodes_from(id_no_dict.keys())
G.add_edges_from(edge_dict.keys())

nx.set_node_attributes(G, id_no_dict, 'line_no')
nx.set_node_attributes(G, id_name_dict, 'station_name')
nx.set_node_attributes(G, id_coord_dict, 'pos')
nx.set_node_attributes(G, {x: id_interchange_dict[x] for x in G.nodes()}, 'is_interchange')
nx.set_edge_attributes(G, edge_dict, 'line_no')

# 그래프를 저장합니다.
nx.write_gml(G, '../data/output/metro_graph.gml')

In [42]:
import pandas as pd
import json

# 역 ID와 연계된 정보에 대한 DataFrame입니다.
station_id_df = pd.DataFrame(nx.get_node_attributes(G, 'line_no').items(), columns=['station_id', 'line_no'])
station_id_df['station_name'] = station_id_df['station_id'].map(nx.get_node_attributes(G, 'station_name'))
station_id_df['is_interchange'] = station_id_df['station_id'].map(nx.get_node_attributes(G, 'is_interchange'))
station_id_df['coord'] = station_id_df['station_id'].map(nx.get_node_attributes(G, 'pos'))
station_id_df['x'], station_id_df['y'] = zip(*station_id_df['coord'])
station_id_df.drop('coord', axis=1, inplace=True)

station_id_df = station_id_df.astype({'is_interchange': bool, 'line_no': 'category'})
station_id_df.to_parquet('../data/output/station_id_table.parquet', compression='gzip', index=False)

# 노선 번호와 연계된 정보에 대한 DataFrame입니다.
line_no_df = pd.DataFrame(no_name_dict.items(), columns=['line_no', 'line_name'])
line_no_df['line_color'] = line_no_df['line_no'].map(no_color_dict)

line_no_df = line_no_df.astype({'line_no': 'category', 'line_name': 'category'})
line_no_df.to_parquet('../data/output/line_no_table.parquet', compression='gzip', index=False)

# 환승역 리스트에 대한 json입니다.
with open('../data/output/interchange_list.json', 'w') as f:
    json.dump(interchange_list, f)